# Libraries

In [198]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from lxml import etree
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import re
from docx import Document
import pandas as pd
import chromedriver_binary

# Text transformations

## All documents LOOP

In [17]:
#Open all documents and store the path in a list
document_list = []
for i in range(9):
    document = Document('data/raw/Podcast_9{}.docx'.format(i))
    document_list.append(document)

In [18]:
#extract headings
def iter_headings(paragraphs):
    for paragraph in paragraphs:
        if paragraph.style.name.startswith('Heading'):
            yield paragraph

In [155]:
#Big LOOP for all documents
results = pd.DataFrame()

for z in document_list:
    
    # Create an empty list for all headings
    heading_list = []
    
    # Extraction headings from this document
    for heading in iter_headings(z.paragraphs):
        heading_list.append(heading.text)

    # Remove the intro and outro
    heading_list = heading_list[1:-1]
    
    text_lines = []

    for i,para in enumerate(z.paragraphs):
        if para.text in heading_list:
            text_lines.append(i)

    #extracting text lines for games only (without intro and outro)       
    gamesText = []
    for k,j in enumerate(text_lines):
        for y, parar in enumerate(z.paragraphs):
            if y>=text_lines[k]:
                gamesText.append(parar.text)

    #creating list mapping where we get the in-text position of heading (number of line)            
    text_lines2 = [text_lines[n]-text_lines[0] for n in range(1,len(text_lines))]
    text_lines2.insert(0,0)
    
    #Removing timestamp from headings
    heading_list2 = []

    for g in heading_list:
        text = g[:-11]
        heading_list2.append(text)
    
    #Extracting text with respect to headings: list of dialogs related to specific game
    extracted_text = []

    for d,w in zip(text_lines2,text_lines2[1:]):
        text = gamesText[d+1:w]
        extracted_text.append(text) 
    
    for game, text in zip(heading_list, extracted_text):
        for sentence in text:
            results = results.append({'game' : game, 'text': sentence}, ignore_index = True)

In [156]:
resultsOld = results.copy()

In [157]:
# Remove the timestamp once again
results['game'] = results['game'].map(lambda x: str(x)[:-11])

In [158]:
results

,game,text
0,Blackout: Hong Kong,Quinns: Let’s talk briefly about Blackout: Hon...
1,Blackout: Hong Kong,Matt: And the reasons you were excited about i...
2,Blackout: Hong Kong,Quinns: This is really more of a public servic...
3,Blackout: Hong Kong,Matt: I think “exciting to try” is the key thi...
4,Blackout: Hong Kong,Quinns: If you haven’t played Great Western Tr...
...,...,...
3470,Too Many Bones,Quinns: Yup.
3471,Too Many Bones,"Matt: And it’s like, cool! Good!"
3472,Too Many Bones,"Quinns: I mean, yeah, that’s the number one th..."
3473,Too Many Bones,"Matt: Yeah. And also, why do you have to be bo..."


In [159]:
# Split the text 
new = results["text"].str.split(":", n = 1, expand = True) 
  
# Extract the person speaking
results["person"] = new[0] 
  
# And what they are saying
results["text"] = new[1] 

# Drop if the formating was off
results.dropna(inplace = True)

# Just a preacusion
results[results['person'].map(len) < 12]

# Remove the part after their name
results["person"] = results["person"].str.split(" ", n = 1, expand = True)[0]

In [160]:
results.person.unique()

array(['Quinns', 'Matt', 'Quinn', 'Kylie', 'This', 'And', 'Audience'],
      dtype=object)

In [161]:
# Remove "This" and "And"
results = results[(results['person'] != 'This') & (results['person'] != 'And')]

In [162]:
results.person.unique()

array(['Quinns', 'Matt', 'Quinn', 'Kylie', 'Audience'], dtype=object)

In [163]:
games = results.game.unique()

In [164]:
pd.DataFrame(games[0:10])

,0
0,Blackout: Hong Kong
1,Just One
2,Sumer
3,The Quacks of Quedlinburg
4,Schummel Hummel
5,The Thing
6,Pipeline
7,Wingspan
8,Twilight Imperium stream plug
9,Chronicles of Crime


In [165]:
results.replace('Twilight Imperium stream plug', 'Twilight Imperium', inplace = True)

In [167]:
games = results.game.unique()

In [192]:
pd.DataFrame(games)

,0
0,Blackout: Hong Kong
1,Just One
2,Sumer
3,The Quacks of Quedlinburg
4,Schummel Hummel
...,...
61,Silk
62,Pandemic: Rapid Response
63,Rail Pass
64,Crokinole


## Webscrapping

In [190]:
headers = {'User-Agent': 'Text mining'}

# Construct the results table
listhashed = [i.replace(' ', '%20') for i in games]
summaryTable = pd.DataFrame()

for j in listhashed:
    # Load the webpage
    r = requests.get('https://boardgamegeek.com/geeksearch.php?action=search&objecttype=boardgame&q={}'.format(j), headers=headers)
    
    # Convert to BS
    bs = BeautifulSoup(r.text)
    
    # Find the right table
    table = bs.findAll(lambda tag: tag.name=='table' and tag.has_attr('cellpadding'))
    #mydivs = bs.findAll("div", {"class": "table-responsive"})
    errors = bs.findAll(text="No Items Found")
    
    if not errors and len(table) > 0:
        tableHTML = table[0]
        #tableHTML = mydivs[0]
        # Convert the HTML of the table to pandas (very usefull function)
        pandasTable = pd.read_html(str(tableHTML), header=0)[0]

        # Extract all the links
        urls = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tableHTML.find_all('a')]      
        urls = pd.DataFrame(urls)
        if not urls.empty:
            urls = urls[0].get(6)

        # Merge the links into the DF
        pandasTable['link'] = urls
        
        # Save to merge later
        pandasTable['previousGameName'] = j

        # Drop unused columns
        pandasTable = pandasTable.drop(['Unnamed: 1', 'Shop'], axis = 1)

        # Create or extend the existing summary table
        if not pandasTable.empty:
            if summaryTable.empty:
                summaryTable = pandasTable.iloc[[0]]
            else:
                summaryTable = pd.concat([summaryTable, pandasTable.iloc[[0]]],ignore_index=True)
    else:
        next

In [191]:
display(summaryTable)

,Board Game Rank,Title,Geek Rating,Avg Rating,Num Voters,link,previousGameName
0,564.0,Blackout: Hong Kong (2018),6.812,7.57,2390.0,/boardgame/262215/blackout-hong-kong,Blackout:%20Hong%20Kong
1,227.0,Just One (2018),7.190,7.61,5849.0,/boardgame/254640/just-one,Just%20One
2,NaN,Men of Sumer (2001),NaN,5.00,1.0,/boardgame/37829/men-sumer,Sumer
3,101.0,The Quacks of Quedlinburg (2018),7.457,7.75,10902.0,/boardgame/244521/quacks-quedlinburg,The%20Quacks%20of%20Quedlinburg
4,6622.0,Schummel Hummel (2017),5.603,6.66,135.0,/boardgame/235944/schummel-hummel,Schummel%20Hummel
5,1855.0,The Thing: Infection at Outpost 31 (2017),6.124,7.30,945.0,/boardgame/226634/thing-infection-outpost-31,The%20Thing
6,629.0,Pipeline (2019),6.755,7.77,1770.0,/boardgame/256730/pipeline,Pipeline
7,27.0,Wingspan (2019),7.877,8.10,21555.0,/boardgame/266192/wingspan,Wingspan
8,59.0,Twilight Imperium (Third Edition) (2005),7.618,7.86,17672.0,/boardgame/12493/twilight-imperium-third-edition,Twilight%20Imperium
9,113.0,Chronicles of Crime (2018),7.423,7.91,8346.0,/boardgame/239188/chronicles-crime,Chronicles%20of%20Crime


In [202]:
# Loading the firefox webdriver
driver = webdriver.Chrome()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 80


In [201]:
chromedriver_binary.chromedriver_filename

'C:\\Users\\szmar\\Anaconda3\\lib\\site-packages\\chromedriver_binary\\chromedriver.exe'

In [19]:
# Construct the results table
detailedTable = pd.DataFrame(columns=['Title','Average Rating','Number of ratings','Std.dev','Number of Comments','Fans','Page Views','Overall Ranking','All Time Plays','Played This Month', 'Own', 'Previously Owned','For Trade','Want in Trade','Wishlist','Has Parts','Want Parts'])
detailedTable2 = pd.DataFrame()

for j in range(1):
    # Load the webpage
    driver.get('https://boardgamegeek.com{}/stats'.format(summaryTable['link'][j]))
    
    # Get the source code and make it BS
    pageSource = driver.page_source
    bs = BeautifulSoup(pageSource, "lxml")
    
    # Find the relevant table
    data = bs.findAll('div',{'class':'outline-item-description'})
    
    # Loop over and strip spaces
    data2 = []
    for i in data:
        data2.append(i.get_text().strip())

    # Remove the ',' 
    for i in range(len(data2)):
        data2[i] = data2[i].replace(",","")

    # Wrangle the columns
    data2[3] = data2[3][0:4]
    data2[0] = float(data2[0])
    data2[1] = int(data2[1]) 
    data2[2] = float(data2[2])
    data2[3] = float(data2[3]) 

    # Convert to int
    for i in range(len(data2[4:])+1):
        i = i+3
        data2[i] = int(data2[i])

    # Special case - two categories. We want to drop both
    if len(data2) == 19:
        data2.pop(8)
        data2.pop(8)
    else:
        data2.pop(8)
    
    # Insert the data
    data2.insert(0,summaryTable['Title'][j]) 
    detailedTable.loc[len(detailedTable)] = data2

In [21]:
display(detailedTable)

,Title,Average Rating,Number of ratings,Std.dev,Number of Comments,Fans,Page Views,Overall Ranking,All Time Plays,Played This Month,Own,Previously Owned,For Trade,Want in Trade,Wishlist,Has Parts,Want Parts
0,Stay Cool (2019),7.173,102,1.35,1,17,8,7121,6336,23,149,1,0,31,172,0,1


In [ ]:
# twoja kolej Michal